# Case Study

We are in charge of operations at a computer manufacturing facility.  We have to determine the number of computers each machine must produce (we have 4 machines) during the next 10 hours in order to maximize production.  The objective function for our challenge is given by:

<center>$\max \sum_{m}\sum_{t}x_{m,t}$</center>

where $x_{m,t}$ represents the number of computers produced by machine $m$ at hour $t$. Some machines will have dependencies on others and the constraints are (for each $t$):

<center> $2x_{2,t}-8x_{3,t}\le0\space \forall t \space\space\space\space
(1)$

$x_{3,t-2}-2x_{3,t-2}+x_{4,t}\ge1\space\space\forall t>2\space\space\space\space(2)$</center>

The total production capacity (for all $t$) is given by:

<center>$\sum_{m}x_{m,t}\le50\space\space\space\forall t\space\space\space\space(3)$

$x_{1,t}+x_{2,t-1}+x_{3,t}+x_{4,t}\le10,\space\space\forall t>1\space\space\space\space(4)$

$0\le x_{m,t}\le10,\space\space\forall m,\space \forall t \space\space\space\space(5)$</center>

Note: constraint $(2)$ does not exist for $t<2$ and constraint $(4)$ does not exist for $t<1$.




In [1]:
import sys
import os

if 'google.colab' in sys.modules:
    # Install Pyomo and IDAES extensions
    !pip install idaes-pse --pre >/dev/null
    !pip install gurobipy
    !idaes get-extensions --to ./bin >/dev/null
    os.environ['PATH'] += ':bin'


In [2]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import gurobipy as gp
import pandas as pd


### Set up global constants and the variable $x$.

In [3]:
model = pyo.ConcreteModel()

T = 10  # maximum of 10 time periods
M = 4  # 4 machine

model.x = pyo.Var(range(1, M+1), range(1, T+1), within=pyo.NonNegativeIntegers, bounds=(0, 10))
x = model.x

### Set up the model objective.

In [4]:
model.obj = pyo.Objective(expr=sum(x[m, t] for m in range(1, M+1) for t in range(1, T+1)), sense=maximize)

### Set up the constraints

In [5]:
model.C1 = pyo.ConstraintList()
for t in range(1, T+1):
  model.C1.add(expr=2*x[2, t] - 8*x[3, t] <= 0)

model.C2 = pyo.ConstraintList()
for t in range(3, T+1):
  model.C2.add(expr=x[3, t-2] - 2*x[3, t-2] + x[4, t] >= 1)

model.C3 = pyo.ConstraintList()
for t in range(1, T+1):
  model.C3.add(expr=sum(x[m, t] for m in range(1, M+1)) <= 50)

model.C4 = pyo.ConstraintList()
for t in range(2, T+1):
  model.C4.add(expr=x[1, t] + x[2, t-1] + x[3, t] + x[4, t] <= 10)


### Invoke the `Gurobi` solver

In [6]:
opt = SolverFactory('gurobi')
results = opt.solve(model)
model.pprint()


1 Var Declarations
    x : Size=40, Index={1, 2, 3, 4}*{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
        Key     : Lower : Value : Upper : Fixed : Stale : Domain
         (1, 1) :     0 :  10.0 :    10 : False : False : NonNegativeIntegers
         (1, 2) :     0 :  10.0 :    10 : False : False : NonNegativeIntegers
         (1, 3) :     0 :  -0.0 :    10 : False : False : NonNegativeIntegers
         (1, 4) :     0 :  -0.0 :    10 : False : False : NonNegativeIntegers
         (1, 5) :     0 :  -0.0 :    10 : False : False : NonNegativeIntegers
         (1, 6) :     0 :  -0.0 :    10 : False : False : NonNegativeIntegers
         (1, 7) :     0 :  -0.0 :    10 : False : False : NonNegativeIntegers
         (1, 8) :     0 :  -0.0 :    10 : False : False : NonNegativeIntegers
         (1, 9) :     0 :  -0.0 :    10 : False : False : NonNegativeIntegers
        (1, 10) :     0 :  -0.0 :    10 : False : False : NonNegativeIntegers
         (2, 1) :     0 :  -0.0 :    10 : False : False : NonNegativ

In [7]:
var_data = [(v.name, value(v)) for v in model.component_objects(Var, active=True)
            for v in v.values()]

In [8]:
# Convert to DataFrame
df = pd.DataFrame(var_data, columns=["Variable", "Value"])
df.head()

,Variable,Value
0,"x[1,1]",10.0
1,"x[1,2]",10.0
2,"x[1,3]",-0.0
3,"x[1,4]",-0.0
4,"x[1,5]",-0.0
